In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import re

import pandas as pd
df = pd.read_csv('../안정성지표.csv', encoding='cp949')

FileNotFoundError: [Errno 2] No such file or directory: '../안정성지표.csv'

In [ ]:
import pandas as pd
credit_spread = pd.read_csv('credit_spread.csv')

In [ ]:
credit_spread

,Unnamed: 0,Unnamed: 1,등급,3월,6월,9월,1년,1년 6개월,2년,3년,5년
0,2010-01-04,0,국고채,2.36,2.92,3.32,3.50,3.99,4.39,4.44,4.98
1,2010-01-04,1,AAA,0.69,0.67,0.63,0.69,0.65,0.61,0.82,0.69
2,2010-01-04,2,AA+,0.76,0.73,0.69,0.76,0.72,0.71,0.92,0.76
3,2010-01-04,3,AA,0.85,0.78,0.75,0.82,0.79,0.77,1.00,1.03
4,2010-01-04,4,AA-,0.99,0.86,0.82,0.87,0.84,0.83,1.13,1.40
...,...,...,...,...,...,...,...,...,...,...,...
37791,2023-12-29,6,A,0.87,1.09,1.14,1.21,1.30,1.38,1.65,2.16
37792,2023-12-29,7,A-,1.09,1.36,1.42,1.49,1.60,1.71,2.10,2.80
37793,2023-12-29,8,BBB+,1.86,2.42,2.79,3.07,3.51,4.10,4.76,4.91
37794,2023-12-29,9,BBB,2.24,2.93,3.37,3.75,4.31,5.04,5.80,5.96


In [ ]:
credit_spread = credit_spread.iloc[:,[0,2,-2]]
credit_spread

,Unnamed: 0,등급,3년
0,2010-01-04,국고채,4.44
1,2010-01-04,AAA,0.82
2,2010-01-04,AA+,0.92
3,2010-01-04,AA,1.00
4,2010-01-04,AA-,1.13
...,...,...,...
37791,2023-12-29,A,1.65
37792,2023-12-29,A-,2.10
37793,2023-12-29,BBB+,4.76
37794,2023-12-29,BBB,5.80


In [ ]:
credit_spread['Unnamed: 0'] = pd.to_datetime(credit_spread['Unnamed: 0'])
credit_spread['연도'] = credit_spread['Unnamed: 0'].dt.year
credit_spread['월'] = credit_spread['Unnamed: 0'].dt.month


In [ ]:
credit_spread

,Unnamed: 0,등급,3년,연도,월
0,2010-01-04,국고채,4.44,2010,1
1,2010-01-04,AAA,0.82,2010,1
2,2010-01-04,AA+,0.92,2010,1
3,2010-01-04,AA,1.00,2010,1
4,2010-01-04,AA-,1.13,2010,1
...,...,...,...,...,...
37791,2023-12-29,A,1.65,2023,12
37792,2023-12-29,A-,2.10,2023,12
37793,2023-12-29,BBB+,4.76,2023,12
37794,2023-12-29,BBB,5.80,2023,12


# 각 연도와 월별로 등급별 데이터 평균 구하기

In [ ]:

# 각 연도와 월별로 등급별 데이터 평균 구하기
monthly_avg_by_year = credit_spread.groupby(['연도', '월', '등급']).mean()
monthly_avg_by_year = monthly_avg_by_year.drop('Unnamed: 0', axis=1)

monthly_avg_by_year.reset_index(inplace=True)
monthly_avg_by_year

,연도,월,등급,3년
0,2010,1,A,1.567000
1,2010,1,A+,1.265000
2,2010,1,A-,1.935000
3,2010,1,AA,0.973500
4,2010,1,AA+,0.884500
...,...,...,...,...
1843,2023,12,AAA,0.514737
1844,2023,12,BBB,5.802105
1845,2023,12,BBB+,4.755263
1846,2023,12,BBB-,7.172105


# '연도', '등급'으로 그룹화하여 '3년' 열의 평균 계산

In [ ]:
# '연도', '등급'으로 그룹화하여 '3년' 열의 평균 계산
result = monthly_avg_by_year.groupby(['연도', '등급'])['3년'].mean().reset_index()

In [ ]:
result.rename(columns=({'3년':'신용스프레드'}), inplace=True)

In [ ]:
result

,연도,등급,신용스프레드
0,2010,A,1.187648
1,2010,A+,0.980860
2,2010,A-,1.543950
3,2010,AA,0.768015
4,2010,AA+,0.689065
...,...,...,...
149,2023,AAA,0.574573
150,2023,BBB,5.816715
151,2023,BBB+,4.768897
152,2023,BBB-,7.186715


# 산식 적용하기

In [ ]:
selected_columns = ['회사명', '거래소코드', '회계년도', '평가사명 및 등급', '신용등급', '회계년월', '평가일', '증권명', '평가사구분',
                        '증권구분', '자본총계', '부채총계', '자산총계', '유동자산', '비유동자산', '당좌자산', '유동부채', '비유동부채']
df.columns = selected_columns
df['부채비율'] = df['부채총계'] / df['자본총계'] * 100
df['당좌비율'] = df['당좌자산'] / df['부채총계']
df['유동비율'] = df['자산총계'] / df['부채총계']
df['자기자본 비율'] = (df['자산총계'] - df['부채총계']) / df['자본총계'] * 100
df['재무레버리지'] = df['부채총계'] / (df['자산총계'] - df['부채총계'])

In [ ]:
df['신용등급']

0                B
1               B-
2              CCC
3              CCC
4              CCC
            ...   
106904     B+(안정적)
106905     BB(부정적)
106906    BB+(안정적)
106907      B(부정적)
106908     B+(안정적)
Name: 신용등급, Length: 106909, dtype: object

In [ ]:
import pandas as pd
import re

# 여기에 데이터프레임 생성 코드 넣기

# 패턴에 따라 등급을 변경하는 함수
def replace_grade(grade):
    if isinstance(grade, str):  # 항목이 문자열인지 확인
        match = re.search(r'(AAA|AA[\+\-]?|A[\+\-]?|BBB[\+\-]?)', grade)
        if match:
            return match.group()
    return ' '  # 문자열이 아닌 경우 공백으로 변경

# 정규식을 사용하여 특정 패턴을 식별하고 함수를 적용하여 대체
df['신용등급'] = df['신용등급'].str.replace(r'\(.*\)', '', regex=True)  # 괄호와 내부 내용 제거
df['신용등급'] = df['신용등급'].apply(replace_grade)
print(df['신용등급'].unique())

[' ' 'BBB-' 'BBB' 'BBB+' 'A-' 'A' 'A+' 'AA' 'AA-' 'AA+' 'AAA']


In [ ]:
df = df.loc[df['증권명'] != '기업어음', :]
df = df.loc[df['증권명'] != 'CP', :]
df = df.loc[df['증권명'] != '전환사채', :]
df = df.loc[df['증권명'] != '기업어음(CP)', :]

In [ ]:
# '전환사채'를 포함하는 행을 공백으로 처리
df.loc[df['증권명'].str.contains('전환사채', na=False), '증권명'] = ''
# '회사채'를 포함하는 행은 '회사채'로 처리
df.loc[df['증권명'].str.contains('회사채', na=False), '증권명'] = '회사채'

# 처리 후 각 값들의 빈도수 확인
value_counts = df['증권명'].value_counts(dropna=False)
print(value_counts)


증권명
회사채                          31774
NaN                          25722
기업신용평가                        4835
-                             3609
전자단기사채                        2379
                             ...  
업무참조                             1
공공기관 입찰용                         1
기업평가 (제39기)                      1
RD기업신용평가                         1
37기재무제표등(재무기준일2012.12.31)        1
Name: count, Length: 1143, dtype: int64


In [ ]:
df.loc[df['증권명'].str.contains('무보증', na=False), '증권명'] = '무보증사채'
df.loc[df['증권명'].str.contains('기업신용평가', na=False), '증권명'] = '기업신용평가'
df.loc[df['증권명'].str.contains('회 사 채', na=False), '증권명'] = '회사채'
df.loc[df['증권명'].str.contains('ICR', na=False), '증권명'] = '기업신용등급'
df.loc[df['증권명'].str.contains('IssuerRating', na=False), '증권명'] = '기업신용등급'
df.loc[df['증권명'].str.contains('Issuer Rating', na=False), '증권명'] = '기업신용등급'
df.loc[df['증권명'].str.contains('IssuerRating', na=False), '증권명'] = '기업신용등급'
df.loc[df['증권명'].str.contains('기업신용', na=False), '증권명'] = '기업신용등급'
df.loc[df['증권명'].str.contains('회사', na=False), '증권명'] = '회사채'
df.loc[df['증권명'].str.contains('회사전체', na=False), '증권명'] = '회사채'
df.loc[df['증권명'].str.contains('신용등급', na=False), '증권명'] = '기업신용등급'



In [ ]:
to_remove = to_remove.union(df[df['증권명'].str.contains('보증사채', na=False)]['증권명'])

# 삭제 진행
df = df[~df['증권명'].isin(to_remove)]
to_remove = to_remove.union(df[df['증권명'].str.contains('CP', na=False)]['증권명'])

# 삭제 진행
df = df[~df['증권명'].isin(to_remove)]

to_remove = to_remove.union(df[df['증권명'].str.contains('CP', na=False)]['증권명'])

# 삭제 진행
df = df[~df['증권명'].isin(to_remove)]
to_remove = to_remove.union(df[df['증권명'].str.contains('기업어음', na=False)]['증권명'])

# 삭제 진행
df = df[~df['증권명'].isin(to_remove)]

to_remove = to_remove.union(df[df['증권명'].str.contains('사모사채', na=False)]['증권명'])

# 삭제 진행
df = df[~df['증권명'].isin(to_remove)]

to_remove = to_remove.union(df[df['증권명'].str.contains('사모사채', na=False)]['증권명'])

# 삭제 진행
df = df[~df['증권명'].isin(to_remove)]

to_remove = to_remove.union(df[df['증권명'].str.contains('해외사채', na=False)]['증권명'])

# 삭제 진행
df = df[~df['증권명'].isin(to_remove)]

to_remove = to_remove.union(df[df['증권명'].str.contains('단기사채', na=False)]['증권명'])

# 삭제 진행
df = df[~df['증권명'].isin(to_remove)]

to_remove = to_remove.union(df[df['증권명'].str.contains('사모사채', na=False)]['증권명'])

# 삭제 진행
df = df[~df['증권명'].isin(to_remove)]

In [ ]:
to_remove = value_counts[value_counts <= 54].index
df = df[~df['증권명'].isin(to_remove)]

value_counts_after_removal = df['증권명'].value_counts(dropna=False)
print(value_counts_after_removal)

증권명
회사채                       32637
NaN                       25722
기업신용등급                    10222
-                          3609
기업평가                        886
현금흐름등급                      732
                            520
업체평가용                       391
SB                          337
현금흐름                        148
글로벌본드                       114
신용평가                        102
글로벌달러본드                      93
BW                           92
단순신용평가(협력업체로등록을위한신용평가)       84
업무참고용                        82
대고객제출용                       77
신용평가용                        73
국제신용평가                       67
외화표시채권                       59
Name: count, dtype: int64


In [ ]:
df = df.reset_index(drop=True)

# 결측치 처리 시작

유동자산

In [ ]:
df['유동자산'].isna().sum()

11710

In [ ]:
# '유동자산' 열에서 결측치를 가진 행 제거
df = df.dropna(subset=['유동자산'])

# 결측치 제거 후 '유동자산' 열에서 결측치의 개수 확인
missing_values_after_removal = df['유동자산'].isna().sum()
print("결측치 제거 후 남은 결측치 개수:", missing_values_after_removal)

결측치 제거 후 남은 결측치 개수: 0


In [ ]:
# '유동자산' 열에서 결측치를 가진 행 제거
df = df.dropna(subset=['부채비율'])

# 결측치 제거 후 '유동자산' 열에서 결측치의 개수 확인
missing_values_after_removal = df['부채비율'].isna().sum()
print("결측치 제거 후 남은 결측치 개수:", missing_values_after_removal)

결측치 제거 후 남은 결측치 개수: 0


In [ ]:
df['회계년도'] = pd.to_datetime(df['회계년도'])
df = df.reset_index(drop=True)

In [ ]:
df['신용등급'] = df['신용등급'].replace(' ', 'BBB-')

In [ ]:
df['신용등급'].value_counts()

신용등급
BBB-    23303
A        7216
AA-      6276
A-       5739
A+       5062
BBB+     4688
BBB      3992
AA       3851
AA+      3359
AAA       849
Name: count, dtype: int64

In [ ]:
df['연도'] = df['회계년도'].dt.year

In [ ]:
df['연도'].dtype

dtype('int32')

In [ ]:
df.drop('회계년월', axis=1, inplace= True)

In [ ]:
df.drop('평가일', axis=1, inplace= True)

In [ ]:
df['평가사명 및 등급'].isna().sum()

10909

In [ ]:
df.isna().sum()

회사명              0
거래소코드            0
회계년도             0
평가사명 및 등급    10911
신용등급             0
증권명          17755
평가사구분        10808
증권구분         10808
자본총계             0
부채총계             0
자산총계             0
유동자산             0
비유동자산            0
당좌자산             0
유동부채             0
비유동부채            0
부채비율             2
당좌비율             2
유동비율             2
자기자본 비율          2
재무레버리지           2
연도               0
dtype: int64

In [ ]:
result.rename(columns={'등급':'신용등급'}, inplace=True)

In [ ]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연도      154 non-null    int32  
 1   신용등급    154 non-null    object 
 2   신용스프레드  154 non-null    float64
dtypes: float64(1), int32(1), object(1)
memory usage: 3.1+ KB


# 전체 데이터프레임 만들기

In [ ]:
# df와 result 데이터프레임 병합
merged_data = pd.merge(df, result, on=['연도', '신용등급'], how='left')
merged_data

,회사명,거래소코드,회계년도,평가사명 및 등급,신용등급,증권명,평가사구분,증권구분,자본총계,부채총계,...,당좌자산,유동부채,비유동부채,부채비율,당좌비율,유동비율,자기자본 비율,재무레버리지,연도,신용스프레드
0,(주)CMG제약,58820,2012-12-01,NaN,BBB-,NaN,NaN,NaN,31590685.0,25517350.0,...,4674359.0,6793095.0,18724256.0,80.774918,0.183184,2.238008,100.000000,0.807749,2012,6.018549
1,(주)CMG제약,58820,2013-12-01,NaN,BBB-,NaN,NaN,NaN,48997281.0,6915281.0,...,21705184.0,5500075.0,1415206.0,14.113602,3.138728,8.085364,100.000002,0.141136,2013,6.058273
2,(주)CMG제약,58820,2014-12-01,NaN,BBB-,NaN,NaN,NaN,49002644.0,9065682.0,...,8827046.0,8719202.0,346480.0,18.500394,0.973677,6.405290,100.000000,0.185004,2014,6.242679
3,(주)CMG제약,58820,2015-12-01,NaN,BBB-,NaN,NaN,NaN,46806888.0,9725933.0,...,2566167.0,8849994.0,875939.0,20.778850,0.263848,5.812586,100.000000,0.207788,2015,6.239654
4,(주)CMG제약,58820,2016-12-01,NaN,BBB-,NaN,NaN,NaN,101107635.0,9872232.0,...,2148521.0,9136755.0,735476.0,9.764082,0.217633,11.241619,100.000000,0.097641,2016,6.523434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64330,흥아해운(주),3280,2022-12-01,한국기업평가,BBB-,회사채,30.0,10.0,135304736.0,181755200.0,...,60803555.0,65840698.0,115914502.0,134.330257,0.334535,1.744434,100.000001,1.343303,2022,6.814670
64331,흥아해운(주),3280,2022-12-01,한국기업평가,BBB-,회사채,30.0,10.0,135304736.0,181755200.0,...,60803555.0,65840698.0,115914502.0,134.330257,0.334535,1.744434,100.000001,1.343303,2022,6.814670
64332,흥아해운(주),3280,2022-12-01,한국기업평가,BBB-,회사채,30.0,10.0,135304736.0,181755200.0,...,60803555.0,65840698.0,115914502.0,134.330257,0.334535,1.744434,100.000001,1.343303,2022,6.814670
64333,흥아해운(주),3280,2022-12-01,NICE신용평가,BBB-,회사채,10.0,10.0,135304736.0,181755200.0,...,60803555.0,65840698.0,115914502.0,134.330257,0.334535,1.744434,100.000001,1.343303,2022,6.814670


In [ ]:
merged_data[merged_data['회사명'] == '삼성전자(주)'].isna().sum()

회사명          0
거래소코드        0
회계년도         0
평가사명 및 등급    0
신용등급         0
증권명          0
평가사구분        0
증권구분         0
자본총계         0
부채총계         0
자산총계         0
유동자산         0
비유동자산        0
당좌자산         0
유동부채         0
비유동부채        0
부채비율         0
당좌비율         0
유동비율         0
자기자본 비율      0
재무레버리지       0
연도           0
신용스프레드       0
dtype: int64

In [ ]:
# 병합된 데이터 확인
merged_data['회사명'].value_counts()

회사명
(주)동양         774
(주)이마트        662
에스케이렌터카(주)    578
현대건설(주)       537
에스케이가스(주)     505
             ... 
광진실업(주)         1
(주)동인기연         1
(주)에스엠벡셀        1
(주)플라즈맵         1
(주)툴젠           1
Name: count, Length: 2020, dtype: int64

In [ ]:
merged_data['신용등급'].unique()

array(['BBB-', 'BBB', 'BBB+', 'A-', 'A', 'A+', 'AA', 'AA-', 'AA+', 'AAA'],
      dtype=object)

## 범주형 변수 mapping 하기

In [ ]:
category_mapping = {
    'AAA': 0, 'AA+': 1, 'AA': 2, 'AA-': 3, 'A+': 4,
    'A': 5, 'A-': 6, 'BBB+': 7, 'BBB': 8, 'BBB-': 9,
}

# 범주형 값으로 이루어진 컬럼들에 대해 매핑 적용
merged_data['신용등급mapping'] = merged_data['신용등급'].replace(category_mapping)

In [ ]:
merged_data.isna().sum()

회사명                0
거래소코드              0
회계년도               0
평가사명 및 등급      10909
신용등급               0
증권명            17753
평가사구분          10806
증권구분           10806
자본총계               0
부채총계               0
자산총계               0
유동자산               0
비유동자산              0
당좌자산               0
유동부채               0
비유동부채              0
부채비율               0
당좌비율               0
유동비율               0
자기자본 비율            0
재무레버리지             0
연도                 0
신용스프레드             0
신용등급mapping        0
dtype: int64

In [ ]:
## 틀렸다.

In [ ]:
idx = merged_data.loc[merged_data.groupby(['회사명', '연도'])['신용등급mapping'].idxmax()]
idx

,회사명,거래소코드,회계년도,평가사명 및 등급,신용등급,증권명,평가사구분,증권구분,자본총계,부채총계,...,유동부채,비유동부채,부채비율,당좌비율,유동비율,자기자본 비율,재무레버리지,연도,신용스프레드,신용등급mapping
0,(주)CMG제약,58820,2012-12-01,NaN,BBB-,NaN,NaN,NaN,31590685.0,25517350.0,...,6793095.0,18724256.0,80.774918,0.183184,2.238008,100.000000,0.807749,2012,6.018549,9
1,(주)CMG제약,58820,2013-12-01,NaN,BBB-,NaN,NaN,NaN,48997281.0,6915281.0,...,5500075.0,1415206.0,14.113602,3.138728,8.085364,100.000002,0.141136,2013,6.058273,9
2,(주)CMG제약,58820,2014-12-01,NaN,BBB-,NaN,NaN,NaN,49002644.0,9065682.0,...,8719202.0,346480.0,18.500394,0.973677,6.405290,100.000000,0.185004,2014,6.242679,9
3,(주)CMG제약,58820,2015-12-01,NaN,BBB-,NaN,NaN,NaN,46806888.0,9725933.0,...,8849994.0,875939.0,20.778850,0.263848,5.812586,100.000000,0.207788,2015,6.239654,9
4,(주)CMG제약,58820,2016-12-01,NaN,BBB-,NaN,NaN,NaN,101107635.0,9872232.0,...,9136755.0,735476.0,9.764082,0.217633,11.241619,100.000000,0.097641,2016,6.523434,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64297,흥아해운(주),3280,2018-12-01,한국기업평가,BBB-,회사채,30.0,10.0,73148175.0,730550282.0,...,312494268.0,418056014.0,998.726601,0.020906,1.100128,100.000000,9.987266,2018,6.562249,9
64305,흥아해운(주),3280,2019-12-01,한국기업평가,BBB-,NaN,30.0,10.0,30919383.0,400043339.0,...,202347422.0,197695917.0,1293.827044,0.034140,1.077290,100.000000,12.938270,2019,6.366166,9
64315,흥아해운(주),3280,2020-12-01,한국기업평가,BBB-,회사채,30.0,10.0,-26283865.0,313059823.0,...,152820982.0,160238840.0,-1191.072253,0.015692,0.916042,100.000000,-11.910723,2020,6.590891,9
64322,흥아해운(주),3280,2021-12-01,한국기업평가,BBB-,회사채,30.0,10.0,102939788.0,144358895.0,...,26670341.0,117688555.0,140.236247,0.264540,1.713082,100.000000,1.402362,2021,6.387397,9


# 13년간 데이터가 있는 회사들만 추출하기

In [ ]:
idx.reset_index(drop=True, inplace=True)

In [ ]:
idx[idx['회사명'].isin(idx['회사명'].value_counts()[idx['회사명'].value_counts() == 13].index)].value_counts()

회사명       거래소코드  회계년도        평가사명 및 등급         신용등급  증권명     평가사구분  증권구분  자본총계          부채총계          자산총계          유동자산          비유동자산         당좌자산          유동부채          비유동부채         부채비율        당좌비율      유동비율      자기자본 비율     재무레버리지    연도    신용스프레드    신용등급mapping
(주)DB하이텍  990    2010-12-01  한국기업평가(AAA~D)     BBB-  기업신용등급  30.0   40.0  3.980162e+08  9.832350e+08  1.381251e+09  3.110902e+08  1.070161e+09  9.277672e+07  1.383358e+08  8.448991e+08  247.033931  0.094359  1.404803  100.000000  2.470339  2010  6.934648  9              1
삼성전기(주)   9150   2015-12-01  한신정평가주식회사(AAA~D)  AA    회사채     90.0   10.0  4.315383e+09  2.954070e+09  7.269453e+09  2.729971e+09  4.539482e+09  1.035257e+09  1.768254e+09  1.185816e+09  68.454399   0.350451  2.460826  100.000000  0.684544  2015  0.309572  2              1
삼부토건(주)   1470   2018-12-01  한국신용평가(AAA~D)     BBB-  회사채     20.0   10.0  1.675355e+08  2.152542e+08  3.827897e+08  2.458535e+08  1.369362e+08  1.140182e+08  1.385995e+08  7.665472e+07  

In [ ]:
idx[idx['회사명'].isin(idx['회사명'].value_counts()[idx['회사명'].value_counts() == 13].index)]['회사명'].unique()

array(['(주)DB하이텍', '(주)ES큐브', '(주)HJ중공업', '(주)MH에탄올', '(주)SH에너지화학',
       '(주)SIMPAC', '(주)STX', '(주)TCC스틸', '(주)가비아', '(주)강원랜드', '(주)경동나비엔',
       '(주)경동인베스트', '(주)경방', '(주)경인양행', '(주)고영테크놀러지', '(주)골프존뉴딘홀딩스',
       '(주)광명전기', '(주)구영테크', '(주)국동', '(주)국보', '(주)국보디자인', '(주)국순당',
       '(주)그래디언트', '(주)글로벌스탠다드테크놀로지', '(주)금비', '(주)금양', '(주)기산텔레콤',
       '(주)나노신소재', '(주)나노엔텍', '(주)나이스홀딩스', '(주)네오위즈', '(주)네오위즈홀딩스',
       '(주)네패스', '(주)넥센', '(주)노루페인트', '(주)노루홀딩스', '(주)노블엠앤비', '(주)녹십자',
       '(주)녹원씨엔아이', '(주)농심', '(주)누리플렉스', '(주)뉴인텍', '(주)뉴프렉스', '(주)다날',
       '(주)다이나믹디자인', '(주)대교', '(주)대명소노시즌', '(주)대우건설', '(주)대웅', '(주)대한항공',
       '(주)더존비즈온', '(주)덕성', '(주)동국알앤에스', '(주)동국에스엔씨', '(주)동방아그로', '(주)동서',
       '(주)동성케미컬', '(주)동아엘텍', '(주)동아지질', '(주)동양', '(주)동양고속', '(주)동양에스텍',
       '(주)동원에프앤비', '(주)동일기연', '(주)동진쎄미켐', '(주)두산', '(주)듀오백',
       '(주)드림어스컴퍼니', '(주)드림텍', '(주)디비아이엔씨', '(주)디아이', '(주)디아이씨',
       '(주)디알비동일', '(주)디에스이엔', '(주)디에스케이', '(주)디에이치오토웨어', '(주)디엔오토모티브',
       '(주)디엠에스', '

In [ ]:
idx.to_csv('final_datas_final.csv', aindex = False ,encoding= 'utf-8-sig')

In [ ]:
idx = idx.drop(['평가사구분', '증권구분'], axis=1)

In [ ]:
idx

,회사명,거래소코드,회계년도,평가사명 및 등급,신용등급,증권명,자본총계,부채총계,자산총계,유동자산,...,유동부채,비유동부채,부채비율,당좌비율,유동비율,자기자본 비율,재무레버리지,연도,신용스프레드,신용등급mapping
0,(주)CMG제약,58820,2012-12-01,NaN,BBB-,NaN,31590685.0,25517350.0,57108035.0,47109213.0,...,6793095.0,18724256.0,80.774918,0.183184,2.238008,100.000000,0.807749,2012,6.018549,9
1,(주)CMG제약,58820,2013-12-01,NaN,BBB-,NaN,48997281.0,6915281.0,55912563.0,38209699.0,...,5500075.0,1415206.0,14.113602,3.138728,8.085364,100.000002,0.141136,2013,6.058273,9
2,(주)CMG제약,58820,2014-12-01,NaN,BBB-,NaN,49002644.0,9065682.0,58068326.0,28300941.0,...,8719202.0,346480.0,18.500394,0.973677,6.405290,100.000000,0.185004,2014,6.242679,9
3,(주)CMG제약,58820,2015-12-01,NaN,BBB-,NaN,46806888.0,9725933.0,56532821.0,26271614.0,...,8849994.0,875939.0,20.778850,0.263848,5.812586,100.000000,0.207788,2015,6.239654,9
4,(주)CMG제약,58820,2016-12-01,NaN,BBB-,NaN,101107635.0,9872232.0,110979867.0,78772201.0,...,9136755.0,735476.0,9.764082,0.217633,11.241619,100.000000,0.097641,2016,6.523434,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17907,흥아해운(주),3280,2018-12-01,한국기업평가,BBB-,회사채,73148175.0,730550282.0,803698457.0,108354984.0,...,312494268.0,418056014.0,998.726601,0.020906,1.100128,100.000000,9.987266,2018,6.562249,9
17908,흥아해운(주),3280,2019-12-01,한국기업평가,BBB-,NaN,30919383.0,400043339.0,430962722.0,93810124.0,...,202347422.0,197695917.0,1293.827044,0.034140,1.077290,100.000000,12.938270,2019,6.366166,9
17909,흥아해운(주),3280,2020-12-01,한국기업평가,BBB-,회사채,-26283865.0,313059823.0,286775958.0,59094649.0,...,152820982.0,160238840.0,-1191.072253,0.015692,0.916042,100.000000,-11.910723,2020,6.590891,9
17910,흥아해운(주),3280,2021-12-01,한국기업평가,BBB-,회사채,102939788.0,144358895.0,247298683.0,55263611.0,...,26670341.0,117688555.0,140.236247,0.264540,1.713082,100.000000,1.402362,2021,6.387397,9


In [ ]:
idx['증권명'].fillna('회사채', inplace= True)

In [ ]:
idx['평가사명 및 등급'].fillna('ubion', inplace=True)

In [ ]:
idx.isna().sum()

In [ ]:

idx['회사명'].isin(idx['회사명'].value_counts())

In [ ]:
# '회사명' 열에서 각 회사의 등장 횟수를 계산하여 Series로 저장
company_counts = idx['회사명'].value_counts()

# 등장 횟수가 13회인 회사명을 선택하여 해당 행 데이터프레임 출력
selected_rows = idx[idx['회사명'].isin(company_counts[company_counts == 13].index)]

# 선택된 행 데이터프레임 출력
selected_rows



,회사명,거래소코드,회계년도,평가사명 및 등급,신용등급,증권명,자본총계,부채총계,자산총계,유동자산,...,유동부채,비유동부채,부채비율,당좌비율,유동비율,자기자본 비율,재무레버리지,연도,신용스프레드,신용등급mapping
8,(주)DB하이텍,990,2010-12-01,한국기업평가(AAA~D),BBB-,기업신용등급,398016157.0,983234957.0,1.381251e+09,311090151.0,...,138335841.0,844899116.0,247.033931,0.094359,1.404803,100.000000,2.470339,2010,6.934648,9
9,(주)DB하이텍,990,2011-12-01,한신정평가(AAA~D),BBB-,기업신용등급,308061688.0,890982181.0,1.199044e+09,279725559.0,...,794704783.0,96277398.0,289.222002,0.044922,1.345755,100.000000,2.892220,2011,6.679649,9
10,(주)DB하이텍,990,2012-12-01,한국신용평가(AAA~D),BBB-,기업신용등급,290893335.0,866861695.0,1.157755e+09,309144147.0,...,555433290.0,311428406.0,297.999848,0.046048,1.335571,100.000000,2.979998,2012,6.018549,9
11,(주)DB하이텍,990,2013-12-01,한신정평가(AAA~D),BBB-,,204756748.0,853635085.0,1.058392e+09,222960700.0,...,476867874.0,376767210.0,416.902052,0.041261,1.239864,100.000000,4.169021,2013,6.058273,9
12,(주)DB하이텍,990,2014-12-01,한신정평가(AAA~D),BBB-,기업신용등급,120702734.0,864173902.0,9.848766e+08,247582371.0,...,376436760.0,487737141.0,715.952219,0.046714,1.139674,100.000000,7.159522,2014,6.242679,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17907,흥아해운(주),3280,2018-12-01,한국기업평가,BBB-,회사채,73148175.0,730550282.0,8.036985e+08,108354984.0,...,312494268.0,418056014.0,998.726601,0.020906,1.100128,100.000000,9.987266,2018,6.562249,9
17908,흥아해운(주),3280,2019-12-01,한국기업평가,BBB-,회사채,30919383.0,400043339.0,4.309627e+08,93810124.0,...,202347422.0,197695917.0,1293.827044,0.034140,1.077290,100.000000,12.938270,2019,6.366166,9
17909,흥아해운(주),3280,2020-12-01,한국기업평가,BBB-,회사채,-26283865.0,313059823.0,2.867760e+08,59094649.0,...,152820982.0,160238840.0,-1191.072253,0.015692,0.916042,100.000000,-11.910723,2020,6.590891,9
17910,흥아해운(주),3280,2021-12-01,한국기업평가,BBB-,회사채,102939788.0,144358895.0,2.472987e+08,55263611.0,...,26670341.0,117688555.0,140.236247,0.264540,1.713082,100.000000,1.402362,2021,6.387397,9


In [ ]:
selected_rows.head(15)

In [ ]:
selected_rows['회사명'].nunique()


735

In [ ]:
735*13

9555

In [ ]:
selected_rows.to_csv('final_final_final.csv', index=False, encoding='utf-8-sig')